# Optional Lab - Regularized Cost and Gradient

## Goals
In this lab, you will:
- 이전의 linear 와 logistic cost함수에 regularization term 을 추가
- 이전에 과적합된 상태에서 regularization term을 추가되어 적용된 결과 반환

In [1]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from plt_overfit import overfit_example, output
from lab_utils_common import sigmoid
np.set_printoptions(precision=8)

In [2]:
# 최소값(argmin), 최대값(argmax), 혹은 조건(where)에 해당하는 색인(index) 값을 찾기
np.argmin([11,1,2,3])

1

# Adding regularization
<img align="Left" src="./images/C1_W3_LinearGradientRegularized.png"  style=" width:400px; padding: 10px; " >
<img align="Center" src="./images/C1_W3_LogisticGradientRegularized.png"  style=" width:400px; padding: 10px; " >

linear 과 logistic regression의 cost와 gradient 함수:
- Cost
    - 선형 회귀와 로지스틱 회귀의 코스트 함수가 서로 상당히 다르지만 regularization 성분은 동일하다. 
- Gradient
    - 선형 회귀와 로지스틱 회귀의 기울기 함수는 매우 유사하다. 
        - $f_{wb}$ 의 구현만 다를 뿐이다. 

## regularization 을 적용한 Cost 함수
### regularized(정규화된) linear regression(선형 회귀) Cost 함수

regularized 선형 회귀의 cost 함수의 공식은 :
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2  + \frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{1}$$ 
여기서:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{2} $$ 
정규화 되지 않은 cost 함수와 비교해보자: 
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 $$ 

regularization 성분의 차이  <span style="color:blue">
    $\frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2$ </span> 
    
이 성분을 이용하여 파라미터의 크기를 최소화하도록 경사 하강에 영향을 준다. 
주의, $b$ 는 정규화되지 않았다, 실무에서의 표준사항

(1) 과 (2) 의 구현 코드. *standard pattern for this course* 은 이 과정의 표준 모든 `m` 개의 training data에 대하여 `for loop` 적용

In [8]:
def compute_cost_linear_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the cost over all examples
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
      lambda_ (scalar): Controls amount of regularization
    Returns:
      total_cost (scalar):  cost 
    """

    m  = X.shape[0]
    n  = len(w)
    cost = 0.
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b #(n,)(n,) = 스칼라
        cost += (f_wb_i - y[i]) **2 # 스칼라
    cost /= (2*m) # 스칼라

    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2) # 스칼라
    reg_cost = (lambda_ / (2*m)) * reg_cost # 스칼라
    total_cost = cost + reg_cost
    return total_cost

In [9]:
np.random.seed(1)
X_tmp = np.random.rand(5,6)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1,) - 0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)
print("Regularized 비용:", cost_tmp)

Regularized 비용: 0.07917239320214275


### regularization를 활용한 미분 계산 (linear/logistic)
linear 과 logistic regression 의 미분의 거의 동일, $f_{\mathbf{w}b}$의 연산에서만 차이.
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)}  +  \frac{\lambda}{m} w_j \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m 은 training data의 개수      
* $f_{\mathbf{w},b}(x^{(i)})$ 은 모델의 출력 결과, $y^{(i)}$ 는 target(정답/label)

      
* <span style="color:blue"> **linear** </span> 회귀 모델에 대하여
    $f_{\mathbf{w},b}(x) = \mathbf{w} \cdot \mathbf{x} + b$  
* <span style="color:blue"> **logistic** </span> 회귀 모델에 대하여
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    여기서 $g(z)$ 는 sigmoid 함수:  
    $g(z) = \frac{1}{1+e^{-z}}$   
    
regularization 을 더한 것은 <span style="color:blue">$\frac{\lambda}{m} w_j $</span>.

In [16]:
def compute_gradient_linear_reg(X, y, w, b, lambda_): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
      lambda_ (scalar): Controls amount of regularization
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]                 
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]               
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m
    return dj_db, dj_dw

In [17]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp = compute_gradient_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)

print(f"dj_db: {dj_db_tmp}",)
print(f"Regularized dj_dw: \n {dj_dw_tmp.tolist()}",)

dj_db: 0.6648774569425726
Regularized dj_dw: 
 [0.20266669606324647, 0.43274529026040554, 0.13824219937625334]


**Expected Output**
```
dj_db: 0.6648774569425726
Regularized dj_dw:
 [0.29653214748822276, 0.4911679625918033, 0.21645877535865857]
 ```

In the plot above, try out regularization on the previous example. In particular:
- Categorical (logistic regression)
    - set degree to 6, lambda to 0 (no regularization), fit the data
    - now set lambda to 1 (increase regularization), fit the data, notice the difference.
- Regression (linear regression)
    - try the same procedure.